In [23]:
!pip install sentence_transformers

  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Obtaining dependency information for transformers<5.0.0,>=4.6.0 from https://files.pythonhosted.org/packages/12/dd/f17b11a93a9ca27728e12512d167eb1281c151c4c6881d3ab59eb58f4127/transformers-4.35.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 1.8 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for torch>=1.6.0 from https://files.pythonhosted.org/packages/ee/9d/85614d94b24f751d175110753ae8a8cb10e7e533a9289bb493f2f4db8bcd/torch-2.1.1-cp39-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/a7/2e/5677e79d26e62f7cf0b98c3781f82cdd51dfb0bf71e7bb03ad176c5f1adc/torchvision-0.16.1-cp39-cp39-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.3 MB/s eta 0:00:00 0:00:01
  Obtaining dependency information for huggingface-hub>=0.4.0 from https://files.pythonhosted.org/p

In [1]:
import pandas as pd
%load_ext autoreload
import numpy as np

In [2]:
news_df = pd.read_csv('./data/war-news.csv', engine='python')

In [3]:
news_df

,Unnamed: 0,Headlines,Summary,Press,Date,Keyword
0,0,I served in Iraq and Afghanistan but the horro...,A WAR hero traumatised by the horrors of comba...,The Sun,1 day ago,Afghanistan
1,1,The forever war in Afghanistan is nowhere near...,Islamic State is seeking to overthrow the Tali...,ThePrint,2 weeks ago,Afghanistan
2,2,"Hell at Abbey Gate: Chaos, Confusion and Death...","In firsthand accounts, Afghan civilians and U....",ProPublica,1 month ago,Afghanistan
3,3,‘A second Afghanistan’: Doubts over Russia’s w...,Russia's lack of progress in its war against U...,Al Jazeera,5 days ago,Afghanistan
4,4,Afghanistan: Former army general vows new war ...,Lt Gen Sami Sadat tells the BBC of planned ope...,BBC,1 week ago,Afghanistan
...,...,...,...,...,...,...
5649,216,"We first cut rations in half. Now, we'll take ...","We first cut rations in half. Now, we'll take ...",USA Today,1 month ago,Yemen
5650,217,Yemen war: Two foreign Doctors Without Borders...,The conflict began in 2014 after Houthi rebels...,Middle East Eye,6 Mar 2022,Yemen
5651,218,"Devastated by war, Yemen's still surviving oil...",Yemen's oil and gas industry could be at a cro...,S&P Global,12 Aug 2021,Yemen
5652,219,Deconstructed: Gas Price Politics And Genocide...,Since we last covered the war in Yemen on this...,The Intercept,11 Feb 2022,Yemen


# Exploratory Data Analysis

### Removing the unnamed columns

In [4]:
news_df = news_df.loc[:, ~news_df.columns.str.contains('^Unnamed')]

In [5]:
news_df.head()

,Headlines,Summary,Press,Date,Keyword
0,I served in Iraq and Afghanistan but the horro...,A WAR hero traumatised by the horrors of comba...,The Sun,1 day ago,Afghanistan
1,The forever war in Afghanistan is nowhere near...,Islamic State is seeking to overthrow the Tali...,ThePrint,2 weeks ago,Afghanistan
2,"Hell at Abbey Gate: Chaos, Confusion and Death...","In firsthand accounts, Afghan civilians and U....",ProPublica,1 month ago,Afghanistan
3,‘A second Afghanistan’: Doubts over Russia’s w...,Russia's lack of progress in its war against U...,Al Jazeera,5 days ago,Afghanistan
4,Afghanistan: Former army general vows new war ...,Lt Gen Sami Sadat tells the BBC of planned ope...,BBC,1 week ago,Afghanistan


In [6]:
news_df.sample()

,Headlines,Summary,Press,Date,Keyword
5225,How refugee stories reveal the true toll of wa...,"A refugee from the Vietnam War, Nguyen calls h...",NPR,1 month ago,Vietnam


In [7]:
news_df.dtypes

Headlines    object
Summary      object
Press        object
Date         object
Keyword      object
dtype: object

In [48]:
news_df.Headlines.apply(lambda x: len(x) if x is not None else 0).describe()

count    5654.000000
mean       60.556420
std        16.249195
min         0.000000
25%        53.000000
50%        60.000000
75%        66.000000
max       149.000000
Name: Headlines, dtype: float64

In [8]:
print(news_df.isnull())

      Headlines  Summary  Press   Date  Keyword
0         False    False  False  False    False
1         False    False  False  False    False
2         False    False  False  False    False
3         False    False  False  False    False
4         False    False  False  False    False
...         ...      ...    ...    ...      ...
5649      False    False  False  False    False
5650      False    False  False  False    False
5651      False    False  False  False    False
5652      False    False  False  False    False
5653       True     True   True   True     True

[5654 rows x 5 columns]


In [9]:
news_df_filtered = news_df[(news_df['Headlines'] != '') & (news_df['Summary'] != '')]
news_df_filtered

,Headlines,Summary,Press,Date,Keyword
0,I served in Iraq and Afghanistan but the horro...,A WAR hero traumatised by the horrors of comba...,The Sun,1 day ago,Afghanistan
1,The forever war in Afghanistan is nowhere near...,Islamic State is seeking to overthrow the Tali...,ThePrint,2 weeks ago,Afghanistan
2,"Hell at Abbey Gate: Chaos, Confusion and Death...","In firsthand accounts, Afghan civilians and U....",ProPublica,1 month ago,Afghanistan
3,‘A second Afghanistan’: Doubts over Russia’s w...,Russia's lack of progress in its war against U...,Al Jazeera,5 days ago,Afghanistan
4,Afghanistan: Former army general vows new war ...,Lt Gen Sami Sadat tells the BBC of planned ope...,BBC,1 week ago,Afghanistan
...,...,...,...,...,...
5649,"We first cut rations in half. Now, we'll take ...","We first cut rations in half. Now, we'll take ...",USA Today,1 month ago,Yemen
5650,Yemen war: Two foreign Doctors Without Borders...,The conflict began in 2014 after Houthi rebels...,Middle East Eye,6 Mar 2022,Yemen
5651,"Devastated by war, Yemen's still surviving oil...",Yemen's oil and gas industry could be at a cro...,S&P Global,12 Aug 2021,Yemen
5652,Deconstructed: Gas Price Politics And Genocide...,Since we last covered the war in Yemen on this...,The Intercept,11 Feb 2022,Yemen


In [10]:
news_df_filtered.dropna(inplace = True)

In [11]:
news_df_filtered

,Headlines,Summary,Press,Date,Keyword
0,I served in Iraq and Afghanistan but the horro...,A WAR hero traumatised by the horrors of comba...,The Sun,1 day ago,Afghanistan
1,The forever war in Afghanistan is nowhere near...,Islamic State is seeking to overthrow the Tali...,ThePrint,2 weeks ago,Afghanistan
2,"Hell at Abbey Gate: Chaos, Confusion and Death...","In firsthand accounts, Afghan civilians and U....",ProPublica,1 month ago,Afghanistan
3,‘A second Afghanistan’: Doubts over Russia’s w...,Russia's lack of progress in its war against U...,Al Jazeera,5 days ago,Afghanistan
4,Afghanistan: Former army general vows new war ...,Lt Gen Sami Sadat tells the BBC of planned ope...,BBC,1 week ago,Afghanistan
...,...,...,...,...,...
5648,The Shattering of Yemen,U.S. President Joe Biden has made ending Yemen...,Foreign Affairs,19 Aug 2021,Yemen
5649,"We first cut rations in half. Now, we'll take ...","We first cut rations in half. Now, we'll take ...",USA Today,1 month ago,Yemen
5650,Yemen war: Two foreign Doctors Without Borders...,The conflict began in 2014 after Houthi rebels...,Middle East Eye,6 Mar 2022,Yemen
5651,"Devastated by war, Yemen's still surviving oil...",Yemen's oil and gas industry could be at a cro...,S&P Global,12 Aug 2021,Yemen


In [41]:
news_df_filtered.Headlines.apply(lambda x: len(x) if x is not None else 0).describe()

count    5653.000000
mean       60.567132
std        16.230654
min         5.000000
25%        53.000000
50%        60.000000
75%        66.000000
max       149.000000
Name: Headlines, dtype: float64

In [42]:
news_df_filtered.Summary.apply(lambda x: len(x) if x is not None else 0).describe()

count    5653.000000
mean      140.290819
std        11.609855
min         7.000000
25%       139.000000
50%       142.000000
75%       144.000000
max       167.000000
Name: Summary, dtype: float64

In [43]:
feature = news_df_filtered.Headlines + news_df_filtered.Summary

In [15]:
feature

0       I served in Iraq and Afghanistan but the horro...
1       The forever war in Afghanistan is nowhere near...
2       Hell at Abbey Gate: Chaos, Confusion and Death...
3       ‘A second Afghanistan’: Doubts over Russia’s w...
4       Afghanistan: Former army general vows new war ...
                              ...                        
5649    We first cut rations in half. Now, we'll take ...
5650    Yemen war: Two foreign Doctors Without Borders...
5651    Devastated by war, Yemen's still surviving oil...
5652    Deconstructed: Gas Price Politics And Genocide...
5653                                                  NaN
Length: 5654, dtype: object

In [44]:
feature.head()

0    I served in Iraq and Afghanistan but the horro...
1    The forever war in Afghanistan is nowhere near...
2    Hell at Abbey Gate: Chaos, Confusion and Death...
3    ‘A second Afghanistan’: Doubts over Russia’s w...
4    Afghanistan: Former army general vows new war ...
dtype: object

In [45]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2') # use multilingual models for texts with non-english characters

embeddings = model.encode(feature, show_progress_bar=True, batch_size=100)

Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:31<00:00,  1.78it/s]


In [46]:
embeddings

array([[ 0.06312142,  0.2201719 , -0.08592805, ..., -0.47542074,
        -0.08063364, -0.3457433 ],
       [-0.11413972,  0.22812489,  0.07895491, ..., -0.32987732,
         0.1889345 , -0.07485486],
       [ 0.07695305, -0.04635577, -0.05303844, ...,  0.03529381,
        -0.16161337, -0.20239653],
       ...,
       [-0.33954358,  0.3634299 , -0.09503815, ...,  0.08140877,
        -0.02458111, -0.11791256],
       [-0.20565271,  0.0162073 ,  0.31460762, ..., -0.10736073,
         0.19337608, -0.2619659 ],
       [-0.2740672 ,  0.21257839,  0.06779844, ..., -0.00420717,
        -0.25486255, -0.25594237]], dtype=float32)

In [47]:
embeddings_list = embeddings.tolist()

In [48]:
import chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 1.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=e622a231776f9d56dd5b831cdb2feb754c6d5383483f29bc416837efc1fc569c
  Stored in directory: /Users/prajwalshrestha/Library/Caches/pip/wheels/fa/17/1f/332799f975d1b2d7f9b3f33bbccf65031e794717d24432caee
Successfully built typing


In [49]:
chroma_client = chromadb.Client()

In [52]:
collection = chroma_client.get_or_create_collection('war_news_metadata_db')

In [95]:
news_df_filtered['news_id'] = news_df_filtered.index

In [ ]:
news_df_filtered.news_id = news_df_filtered['news_id'].astype(str)

In [105]:
news_id = news_df_filtered.news_id.to_list()

In [146]:
meta_data = news_df_filtered.to_dict(orient='records')

In [148]:
#collection.delete(ids=news_ids)
collection.add(ids=news_ids, embeddings=embeddings_list, metadatas=meta_data)

In [149]:
current_news = news_df_filtered.sample(1, random_state=2023)

In [150]:
current_news

,Headlines,Summary,Press,Date,Keyword,news_id
1223,French outrage over US submarine deal will not...,"In the U.S., protests erupted in Philadelphia ...",The Conversation,15 Oct 2021,France,1223


In [151]:
current_news_id = current_news.index.values[0]

In [152]:
current_news_id

1223

In [153]:
current_news_meta_data = collection.get(ids=[str(current_news_id)], include=['embeddings'])

In [155]:
query = current_news_meta_data['embeddings']

In [157]:
result = collection.query(query_embeddings=query, n_results=5)

In [158]:
result

{'ids': [['1223', '1164', '1055', '1240', '1242']],
 'distances': [[0.0,
   11.051956176757812,
   11.83742904663086,
   12.476518630981445,
   12.536988258361816]],
 'metadatas': [[{'Date': '15 Oct 2021',
    'Headlines': 'French outrage over US submarine deal will not sink a longstanding alliance',
    'Keyword': 'France',
    'Press': 'The Conversation',
    'Summary': 'In the U.S., protests erupted in Philadelphia demanding war with France. \nAnd Congress soon passed legislation to fund a naval force,...',
    'news_id': 1223},
   {'Date': '17 Sep 2021',
    'Headlines': 'France cancels Washington reception and tones down celebrations of \nUS-French Revolutionary War victory amid submarine spat',
    'Keyword': 'France',
    'Press': 'CNN',
    'Summary': 'France cancels Washington reception and tones down celebrations of \nUS-French Revolutionary War victory amid submarine spat ; 01:06. Why France \nis...',
    'news_id': 1164},
   {'Date': '12 hours ago',
    'Headlines': 'Joint 

# Reranking

In [159]:
reranked_result = collection.query(query_embeddings=query, n_results=5, where={'news_id': {'$ne': int(current_news_id)}})

In [160]:
reranked_result

{'ids': [['1164', '1055', '1240', '1242', '1239']],
 'distances': [[11.051956176757812,
   11.83742904663086,
   12.476518630981445,
   12.536988258361816,
   12.731101989746094]],
 'metadatas': [[{'Date': '17 Sep 2021',
    'Headlines': 'France cancels Washington reception and tones down celebrations of \nUS-French Revolutionary War victory amid submarine spat',
    'Keyword': 'France',
    'Press': 'CNN',
    'Summary': 'France cancels Washington reception and tones down celebrations of \nUS-French Revolutionary War victory amid submarine spat ; 01:06. Why France \nis...',
    'news_id': 1164},
   {'Date': '12 hours ago',
    'Headlines': 'Joint U.S. Forces participate in V-E Day and Marara 22',
    'Keyword': 'France',
    'Press': 'U.S. Army',
    'Summary': '... War in Europe after the Nazi capitulation,” said French Colonel Olivier \n... and this is what France represents with its strategy of a...',
    'news_id': 1055},
   {'Date': '6 May 2021',
    'Headlines': 'Four months int

In [163]:
recommendations = pd.DataFrame(reranked_result['metadatas'][0])

In [164]:
recommendations

,Date,Headlines,Keyword,Press,Summary,news_id
0,17 Sep 2021,France cancels Washington reception and tones ...,France,CNN,France cancels Washington reception and tones ...,1164
1,12 hours ago,Joint U.S. Forces participate in V-E Day and M...,France,U.S. Army,"... War in Europe after the Nazi capitulation,...",1055
2,6 May 2021,"Four months into Brexit, the UK and France hav...",France,CNN,... forgiven for thinking that the UK was prep...,1240
3,16 Sep 2021,U.S. Says It Gave France Hours' Notice of Deal...,France,The New York Times,The degree of French anger recalled the acrimo...,1242
4,8 Sep 2021,France’s Military Is A Powerful – But Fragile ...,France,Forbes,And the French military would be an invaluable...,1239


In [166]:
recommendations['distance'] = reranked_result['distances'][0]

In [167]:
recommendations

,Date,Headlines,Keyword,Press,Summary,news_id,distance
0,17 Sep 2021,France cancels Washington reception and tones ...,France,CNN,France cancels Washington reception and tones ...,1164,11.051956
1,12 hours ago,Joint U.S. Forces participate in V-E Day and M...,France,U.S. Army,"... War in Europe after the Nazi capitulation,...",1055,11.837429
2,6 May 2021,"Four months into Brexit, the UK and France hav...",France,CNN,... forgiven for thinking that the UK was prep...,1240,12.476519
3,16 Sep 2021,U.S. Says It Gave France Hours' Notice of Deal...,France,The New York Times,The degree of French anger recalled the acrimo...,1242,12.536988
4,8 Sep 2021,France’s Military Is A Powerful – But Fragile ...,France,Forbes,And the French military would be an invaluable...,1239,12.731102
